## This notebook shows the steps for re-organizing data in BIDS format (the format needed for prep-processing in fMRIPrep)
- To read more on BIDs format, see [here](https://bids.neuroimaging.io/)
- The script is specific to Time 2 data that Seda sent, but it could be applied to any other data in future

In [ ]:
!ls -l T1_first_visitors_brain_age_part2/ | head -5

#### Import libraries

In [ ]:
from pathlib import Path
import shutil

- Make a bids_raw directory to place all images

In [ ]:
BASE_PATH = Path('__file__').resolve().parent
print(f' BASE_PATH = {BASE_PATH}')
BIDS_DIR = BASE_PATH / 'bids_raw'
BIDS_DIR.mkdir(exist_ok=True, parents=True)
print(f' BIDS_DIR = {BIDS_DIR}')

In [ ]:
#!ls -l 

- List all participant T1w files

In [ ]:
t1_dir = BASE_PATH / 'T1_first_visitors_brain_age_part2'
t1_list = [i for i in t1_dir.glob('*.nii.gz')]
print(f' + found {len(t1_list)} T1 files in {t1_dir}')
!ls {t1_dir}/

- List all participant DWI files

In [ ]:
dwi_dir = BASE_PATH / 'dwi_first_visitors_brain_age_part2'
dwi_list = [i for i in dwi_dir.glob('*.nii.gz')]
print(f' + found {len(dwi_list)} DWI files in {dwi_dir}')
!ls {dwi_dir}/

- There were some incorrectly named files in the DWI folder, so we renamed them using the function below

In [ ]:
## fix naming issues in dwi folder
dwi_dir = BASE_PATH / 'dwi_first_visitors_brain_age_part2'
## fix .nii.gz
ext_list = ['.nii.gz', '.json', '.bval', '.bvec']
for ext in ext_list:
    n_len = len(ext)+4
    for f in dwi_dir.glob(f'*{ext}'):
        #print(f.name)
        if len(f.name) != n_len:
            new_fn = f.with_name(f.name[-n_len:])
            print(f' * incorrectly name file="{f.name}", renaming={new_fn}')
            f.replace(new_fn)
dwi_list = [i for i in dwi_dir.glob('*.nii.gz')]
print(f' + found {len(dwi_list)} DWI files in {dwi_dir}')
!ls {dwi_dir}/   

- List functional files

In [ ]:
func_dir = BASE_PATH / 'rsfmri_first_visitors_brain_age_part2'
## fix .nii.gz
ext_list = ['.nii.gz', '.json']
for ext in ext_list:
    n_len = len(ext)+4
    for f in func_dir.glob(f'*{ext}'):
        #print(f.name)
        if len(f.name) != n_len:
            new_fn = f.with_name(f.name[-n_len:])
            print(f' * incorrectly name file="{f.name}", renaming={new_fn}')
            f.replace(new_fn)
func_list = [i for i in func_dir.glob('*.nii.gz')]
print(f' + found {len(func_list)} FUNC files in {func_dir}')
!ls {func_dir}/

- 33 T1w images, 41 DWI images, and  50 func images. means that there are participants with DWI/Func that do not have a T1w

### Make Folders for each participant with a T1w image
- Since a T1w is necessary for processing in fMRIPrep, we keep only those participants who have a T1w image

In [ ]:
## make folders for each subject with a T1 image
for i,img in enumerate(t1_list):
    ssid = img.name[:-7]
    bid = f'sub-{ssid}'
    print(f' + found t1 = {img.name}, ssid={ssid}, making bids-dir = {bid}')
    sdir = BIDS_DIR / bid
    s_anat_dir = sdir / 'anat'
    s_anat_dir.mkdir(exist_ok=True, parents=True)
    t1_fn = s_anat_dir / f'{bid}_T1w'
    img_bn = img.parent / img.name[:-7]
    for ext in ['.nii.gz','.json']:
        in_fn = Path(f'{img_bn}{ext}')
        out_fn = Path(f'{t1_fn}{ext}')
        if not in_fn.exists():
            raise FileNotFound(f'missing {in_fn}')
        if not out_fn.exists():
            print(f' ++ copying {in_fn} to {out_fn}')
            shutil.copy2(in_fn, out_fn)
    ## add dwi
    d_fn = dwi_dir / img.name
    d_bval = dwi_dir/ f'{img.name[:-7]}.bval'
    d_bvec = dwi_dir/ f'{img.name[:-7]}.bvec'
    d_json = dwi_dir/ f'{img.name[:-7]}.json'
    if d_fn.exists() and d_bval.exists() and d_bvec.exists() and d_json.exists():
        print(f' + found dwi-file={d_fn}')
        d_bn = dwi_dir/ img.name[:-7] ## 
        s_dwi_dir = sdir / 'dwi'
        s_dwi_dir.mkdir(exist_ok=True, parents=True)
        d_out_fn = s_dwi_dir / f'{bid}_dwi.nii.gz'
        d_bval_out_fn = s_dwi_dir / f'{bid}_dwi.bval'
        d_bvec_out_fn = s_dwi_dir / f'{bid}_dwi.bvec'
        d_json_out_fn = s_dwi_dir / f'{bid}_dwi.json'
        print(f' ++ copying {d_fn} to {d_out_fn}')
        shutil.copy2(d_fn, d_out_fn)
        shutil.copy2(d_bval, d_bval_out_fn)
        shutil.copy2(d_bvec, d_bvec_out_fn)
        shutil.copy2(d_json, d_json_out_fn)
    ## add funcs
    func_fn = func_dir / img.name
    func_json = func_dir/ f'{img.name[:-7]}.json'
    if func_fn.exists() and func_json.exists():
        print(f' + found func-file={func_fn}')
        s_func_dir = sdir / 'func'
        s_func_dir.mkdir(exist_ok=True, parents=True)
        func_out_fn = s_func_dir / f'{bid}_task-rest_bold.nii.gz'
        func_json_out_fn = s_func_dir / f'{bid}_task-rest_bold.json'
        print(f' ++ copying {func_fn} to {func_out_fn}')
        shutil.copy2(func_fn, func_out_fn)
        shutil.copy2(func_json, func_json_out_fn)

#### Confirm that files were copied and organized according to BIDS format

In [ ]:
!tree -lsh -C {BIDS_DIR}/